In [ ]:
import requests
import zipfile
import torch
from torch.utils.data import Dataset
from pathlib import Path
from PIL import Image

In [ ]:
url = "https://firebasestorage.googleapis.com/v0/b/grandmacan-2dae4.appspot.com/o/ML_data%2Fone_piece_full.zip?alt=media&token=937656fd-f5c1-44f5-b174-1e2d590b8ef3"

with open("one_piece_full.zip", "wb") as f:
  req = requests.get(url)
  f.write(req.content)

with zipfile.ZipFile("one_piece_full.zip", "r") as zip_file:
  zip_file.extractall("one_piece_full")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def accuracy_fn(y_pred, y_true):
  correct_num = (y_pred==y_true).sum()
  acc = correct_num / len(y_true) * 100
  return acc

def train_step(dataloader, model, cost_fn, optimizer, accuracy_fn, device):
  train_cost = 0
  train_acc = 0
  for batch, (x, y) in enumerate(dataloader):
    x = x.to(device)
    y = y.to(device)
    model.to(device)

    model.train()

    y_pred = model(x)

    cost = cost_fn(y_pred, y)

    train_cost += cost
    train_acc += accuracy_fn(y_pred.argmax(dim=1), y)

    optimizer.zero_grad()

    cost.backward()

    optimizer.step()

  train_cost /= len(train_dataloader)
  train_acc /= len(train_dataloader)

  print(f"\nTrain Cost: {train_cost:.4f}, Train Acc: {train_acc:.2f}")


def test_step(dataloader, model, cost_fn, accuracy_fn, device):
  test_cost = 0
  test_acc = 0
  model.eval()
  with torch.inference_mode():
    for x, y in dataloader:
      x = x.to(device)
      y = y.to(device)
      model.to(device)

      test_pred = model(x)

      test_cost += cost_fn(test_pred, y)
      test_acc += accuracy_fn(test_pred.argmax(dim=1), y)

    test_cost /= len(test_dataloader)
    test_acc /= len(test_dataloader)

  print(f"Test Cost: {test_cost:.4f}, Test Acc: {test_acc:.2f} \n")

In [ ]:
class ImageDataset(Dataset):
  def __init__(self, root, train, transform=None):

    if train:
      image_root = Path(root) / "train"
    else:
      image_root = Path(root) / "test"

    with open(Path(root) / "classnames.txt", "r") as f:
      lines = f.readlines()
      self.classes = [line.strip() for line in lines]

    self.paths = [i for i in image_root.rglob("*") if i.is_file()]
    self.transform = transform

  def __getitem__(self, index):
    img = Image.open(self.paths[index]).convert("RGB")
    class_name = self.paths[index].parent.name
    class_idx = self.classes.index(class_name)

    if self.transform:
      return self.transform(img), class_idx
    else:
      return img, class_idx


  def __len__(self):
    return len(self.paths)

In [ ]:
import torchvision
# EfficientNet_B0 在更新後似乎出了點問題, 改用 EfficientNet_B1
# weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
# model = torchvision.models.efficientnet_b0(weights=weights)

weights = torchvision.models.EfficientNet_B1_Weights.DEFAULT
model = torchvision.models.efficientnet_b1(weights=weights)
model

In [ ]:
efficientnet_b1_transforms = weights.transforms()

In [ ]:
train_dataset = ImageDataset(root="one_piece_full",
              train=True,
              transform=efficientnet_b1_transforms
)

test_dataset = ImageDataset(root="one_piece_full",
              train=False,
              transform=efficientnet_b1_transforms
)

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 16

train_dataloader = DataLoader(dataset=train_dataset,
                batch_size=BATCH_SIZE,
                shuffle=True
)

test_dataloader = DataLoader(dataset=test_dataset,
                batch_size=BATCH_SIZE,
                shuffle=False
)

In [ ]:
len(train_dataloader), len(test_dataloader)

(189, 47)

In [ ]:
!pip install torchinfo
from torchinfo import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
summary(model=model,
    input_size=(16, 3, 64, 64),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    row_settings=["var_names"]
)

/usr/local/lib/python3.9/dist-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/usr/local/lib/python3.9/dist-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type (var_name))                                      Input Shape               Output Shape              Param #                   Trainable
EfficientNet (EfficientNet)                                  [16, 3, 64, 64]           [16, 18]                  --                        Partial
├─Sequential (features)                                      [16, 3, 64, 64]           [16, 1280, 2, 2]          --                        False
│    └─Conv2dNormActivation (0)                              [16, 3, 64, 64]           [16, 32, 32, 32]          --                        False
│    │    └─Conv2d (0)                                       [16, 3, 64, 64]           [16, 32, 32, 32]          (864)                     False
│    │    └─BatchNorm2d (1)                                  [16, 32, 32, 32]          [16, 32, 32, 32]          (64)                      False
│    │    └─SiLU (2)                                         [16, 32, 32, 32]          [16, 32, 32, 32]          --         

In [ ]:
from torch import nn
model.classifier[1] = nn.Linear(in_features=1280, out_features=18, bias=True)

In [ ]:
for param in model.features.parameters():
  param.requires_grad=False

In [ ]:
cost_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [ ]:
from tqdm.auto import tqdm

epochs = 10

for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n-------")

  train_step(train_dataloader, model, cost_fn, optimizer, accuracy_fn, device)

  test_step(test_dataloader, model, cost_fn, accuracy_fn, device)


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
-------


/usr/local/lib/python3.9/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



Train Cost: 1.9935, Train Acc: 50.11
Test Cost: 1.4989, Test Acc: 67.08 

Epoch: 1
-------

Train Cost: 1.2094, Train Acc: 70.95
Test Cost: 1.1799, Test Acc: 70.67 

Epoch: 2
-------

Train Cost: 0.9792, Train Acc: 75.93
Test Cost: 1.0750, Test Acc: 73.21 

Epoch: 3
-------

Train Cost: 0.8369, Train Acc: 79.11
Test Cost: 0.9915, Test Acc: 73.35 

Epoch: 4
-------

Train Cost: 0.7357, Train Acc: 81.35
Test Cost: 0.9155, Test Acc: 75.08 

Epoch: 5
-------

Train Cost: 0.6885, Train Acc: 82.12
Test Cost: 0.9085, Test Acc: 75.61 

Epoch: 6
-------

Train Cost: 0.6318, Train Acc: 83.36
Test Cost: 0.8843, Test Acc: 75.99 

Epoch: 7
-------

Train Cost: 0.5847, Train Acc: 83.70
Test Cost: 0.8339, Test Acc: 77.60 

Epoch: 8
-------

Train Cost: 0.5576, Train Acc: 84.90
Test Cost: 0.8327, Test Acc: 77.87 

Epoch: 9
-------

Train Cost: 0.5225, Train Acc: 85.80
Test Cost: 0.8067, Test Acc: 76.94 

